In [ ]:
# INSTALL LIBRARIES
!pip install gradio xgboost --quiet

In [1]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gradio as gr

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

# LOAD DATA
df = pd.read_csv("dynamic_pricing.csv")

# CLEANING AND FEATURE ENGINEERING
df.dropna(inplace=True)
df = df[df['Number_of_Drivers'] > 0]

df['demand_supply_ratio'] = df['Number_of_Riders'] / df['Number_of_Drivers']
df['is_peak'] = df['Time_of_Booking'].apply(lambda x: 1 if x in ['Morning', 'Evening'] else 0)

# Estimate Base Fare from Historical Cost Pattern
base_fare_model = np.polyfit(df['Expected_Ride_Duration'], df['Historical_Cost_of_Ride'], 1)
slope, intercept = base_fare_model
df['base_fare'] = df['Expected_Ride_Duration'] * slope + intercept

# Surge Multiplier
def surge(row):
    multiplier = 1.0
    if row['demand_supply_ratio'] > 1.2:
        multiplier += 0.15
    if row['is_peak'] == 1:
        multiplier += 0.10
    if row['Vehicle_Type'] == 'Luxury':
        multiplier += 0.25
    elif row['Vehicle_Type'] == 'Premium':
        multiplier += 0.15
    return multiplier

df['surge_multiplier'] = df.apply(surge, axis=1)
df['Dynamic_Fare'] = df['base_fare'] * df['surge_multiplier']

# Encode Categorical Features
categorical = ['Location_Category', 'Customer_Loyalty_Status', 'Time_of_Booking', 'Vehicle_Type']
ohe = OneHotEncoder(drop='first', sparse_output=False)
encoded = ohe.fit_transform(df[categorical])
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out())

# Numerical Features
X_numeric = df[['Number_of_Riders', 'Number_of_Drivers', 'Number_of_Past_Rides',
                'Average_Ratings', 'Expected_Ride_Duration', 'demand_supply_ratio', 'is_peak']]
X = pd.concat([X_numeric.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
y = df['Dynamic_Fare']

# Train-Test Split & Scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train[X_numeric.columns] = scaler.fit_transform(X_train[X_numeric.columns])
X_test[X_numeric.columns] = scaler.transform(X_test[X_numeric.columns])

# Model Training
model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Compare to base fare
y_base = X_test['Expected_Ride_Duration'] * slope + intercept
rmse_base = np.sqrt(mean_squared_error(y_test, y_base))
improvement = 100 * (rmse_base - rmse) / rmse_base

print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.2f}")
print(f"Base Fare RMSE: {rmse_base:.2f}, Improvement: {improvement:.2f}%")

# Gradio Prediction Function

def predict_fare(Number_of_Riders, Number_of_Drivers, Location_Category, Customer_Loyalty_Status,
                 Number_of_Past_Rides, Average_Ratings, Time_of_Booking, Vehicle_Type, Expected_Ride_Duration):

    demand_supply_ratio = Number_of_Riders / Number_of_Drivers if Number_of_Drivers > 0 else 0
    is_peak = 1 if Time_of_Booking in ['Morning', 'Evening'] else 0

    input_dict = {
        'Number_of_Riders': Number_of_Riders,
        'Number_of_Drivers': Number_of_Drivers,
        'Number_of_Past_Rides': Number_of_Past_Rides,
        'Average_Ratings': Average_Ratings,
        'Expected_Ride_Duration': Expected_Ride_Duration,
        'demand_supply_ratio': demand_supply_ratio,
        'is_peak': is_peak
    }

    for col in ohe.get_feature_names_out():
        input_dict[col] = 0

    selected = {
        f'Location_Category_{Location_Category}',
        f'Customer_Loyalty_Status_{Customer_Loyalty_Status}',
        f'Time_of_Booking_{Time_of_Booking}',
        f'Vehicle_Type_{Vehicle_Type}'
    }

    for feature in selected:
        if feature in input_dict:
            input_dict[feature] = 1

    input_df = pd.DataFrame([input_dict])
    input_df[X_numeric.columns] = scaler.transform(input_df[X_numeric.columns])
    prediction = model.predict(input_df)[0]
    return round(prediction, 2)

MAE: 5.16, RMSE: 9.92, R²: 1.00
Base Fare RMSE: 515.61, Improvement: 98.08%


In [3]:
# Gradio Interface
import gradio as gr
import pandas as pd

def predict_fare(Number_of_Riders, Number_of_Drivers , Location_Category, Customer_Loyalty_Status,
                 Number_of_Past_Rides, Average_Ratings, Time_of_Booking, Vehicle_Type, Expected_Ride_Duration):

    demand_supply_ratio = Number_of_Riders / Number_of_Drivers if Number_of_Drivers > 0 else 0
    is_peak = 1 if Time_of_Booking in ['Morning', 'Evening'] else 0

    input_dict = {
        'Number_of_Riders': Number_of_Riders,
        'Number_of_Drivers': Number_of_Drivers,
        'Number_of_Past_Rides': Number_of_Past_Rides,
        'Average_Ratings': Average_Ratings,
        'Expected_Ride_Duration': Expected_Ride_Duration,
        'demand_supply_ratio': demand_supply_ratio,
        'is_peak': is_peak
    }

    for col in ohe.get_feature_names_out():
        input_dict[col] = 0

    selected = {
        f'Location_Category_{Location_Category}',
        f'Customer_Loyalty_Status_{Customer_Loyalty_Status}',
        f'Time_of_Booking_{Time_of_Booking}',
        f'Vehicle_Type_{Vehicle_Type}'
    }

    for feature in selected:
        if feature in input_dict:
            input_dict[feature] = 1

    input_df = pd.DataFrame([input_dict])
    input_df[X_numeric.columns] = scaler.transform(input_df[X_numeric.columns])
    prediction = model.predict(input_df)[0]
    return round(prediction, 2)


with gr.Blocks(theme=gr.themes.Soft()) as interface:
    gr.Markdown(
        """
        <div style="background-color:#f4f9ff; padding: 10px; border-radius: 12px; text-align: center; box-shadow: 0 4px 12px rgba(0,0,0,0.08);">
            <h2 style="color:#1A5276; margin-bottom: 10px;">🚖 Real-Time Dynamic Ride Fare Predictor</h2>
            <p style="color:#1A5276; font-size: 16px;">
                Enter ride details below to get a predicted fare based on demand-supply, loyalty, and ride context.
            </p>
        </div>
        """
    )

    with gr.Row():
        with gr.Column():
            num_riders = gr.Number(label="Number of Riders")
            num_drivers = gr.Number(label="Number of Drivers")
            location = gr.Dropdown(['Urban', 'Suburban', 'Rural'], label="Location Category")
            loyalty = gr.Dropdown(['Silver', 'Regular', 'Gold'], label="Customer Loyalty Status")
        with gr.Column():
            past_rides = gr.Number(label="Number of Past Rides")
            rating = gr.Slider(1.0, 5.0, step=0.1, label="Average Ratings")
            booking_time = gr.Dropdown(['Morning', 'Afternoon', 'Evening', 'Night'], label="Time of Booking")
            vehicle = gr.Dropdown(['Economy', 'Premium', 'Luxury'], label="Vehicle Type")
            duration = gr.Number(label="Expected Ride Duration (minutes)")

    fare_output = gr.Number(label="Predicted Dynamic Fare (₹)", interactive=False)

    with gr.Row():
        predict_btn = gr.Button("💰 Predict Fare", variant="primary")
        clear_btn = gr.Button("🧹 Clear", variant="secondary")

    predict_btn.click(
        fn=predict_fare,
        inputs=[num_riders, num_drivers, location, loyalty, past_rides, rating, booking_time, vehicle, duration],
        outputs=fare_output
    )

    clear_btn.click(
        lambda: [None]*9 + [0.0],
        inputs=[],
        outputs=[num_riders, num_drivers, location, loyalty, past_rides, rating, booking_time, vehicle, duration, fare_output]
    )

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5bdf5705fbd30ed988.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
